In [3]:
import requests
import pandas as pd
from tqdm import tqdm  # 진행 상황 표시

# OSRM 서버 URL
osrm_url = "http://localhost:5000/route/v1/walking/{},{};{},{}?overview=full&geometries=geojson"

# 좌표별로 경로를 요청하고 노드를 가져오는 함수
def get_route(departure_x, departure_y, arrival_x, arrival_y):
    try:
        # URL 구성
        url = osrm_url.format(departure_x, departure_y, arrival_x, arrival_y)
        response = requests.get(url)
        data = response.json()
        if 'routes' in data and len(data['routes']) > 0:
            return data['routes'][0]['geometry']['coordinates']  # 경로의 노드 좌표
        else:
            return None
    except Exception as e:
        print(f"Error retrieving route: {e}")
        return None

# 경로를 분해하여 pandas로 처리 후 새로운 CSV로 저장하는 함수
def save_routes_with_full_info_pandas(file_paths, output_files):
    for file_path, output_file in zip(file_paths, output_files):
        print(f"처리 중: {file_path} -> {output_file}")
        
        # CSV 파일 불러오기
        data = pd.read_csv(file_path)

        # 출발 및 도착 좌표가 유효한 행만 필터링
        valid_data = data.dropna(subset=['DPR_X_AXIS_WGS', 'DPR_Y_AXIS_WGS', 'ARV_X_AXIS_WGS', 'ARV_Y_AXIS_WGS'])

        all_segments = []

        # tqdm을 사용하여 진행 상황을 표시하면서 각 데이터에 대해 경로를 가져옴
        for index, row in tqdm(valid_data.iterrows(), total=valid_data.shape[0], desc=f"{file_path} 경로 처리 중"):
            route = get_route(row['DPR_X_AXIS_WGS'], row['DPR_Y_AXIS_WGS'], row['ARV_X_AXIS_WGS'], row['ARV_Y_AXIS_WGS'])
            if route:
                # 경로의 각 구간을 (A->B, B->C) 형식으로 분해하여 리스트에 추가
                for i in range(len(route) - 1):
                    segment = {
                        '출발시간': row['DPR_MT10_UNIT_TM'],
                        '유동인구': row['POPL_CNT'],
                        '노드X좌표': route[i][0],
                        '노드Y좌표': route[i][1],
                        '다음노드X좌표': route[i+1][0],
                        '다음노드Y좌표': route[i+1][1]
                    }
                    all_segments.append(segment)

        # 분해된 경로를 pandas DataFrame으로 변환
        segments_df = pd.DataFrame(all_segments)

        # 새로운 CSV 파일로 저장
        segments_df.to_csv(output_file, index=False)
        print(f"파일 저장 완료: {output_file}")

# 지역별 파일 경로 및 출력 파일 경로 설정
file_paths = [
    r'C:\Users\pc10\경로분해\1번지역.csv',
    r'C:\Users\pc10\경로분해\2번지역.csv',
    r'C:\Users\pc10\경로분해\3번지역.csv',
    r'C:\Users\pc10\경로분해\4번지역.csv'
]

output_files = [
    r'C:\Users\pc10\경로분해\경로분해_1.csv',
    r'C:\Users\pc10\경로분해\경로분해_2.csv',
    r'C:\Users\pc10\경로분해\경로분해_3.csv',
    r'C:\Users\pc10\경로분해\경로분해_4.csv'
]

# 처리 실행
save_routes_with_full_info_pandas(file_paths, output_files)

print("모든 지역 경로 분해 완료!")


경로 처리 중: 100%|████████████████████████████████████████████████████████████████████| 7774/7774 [01:13<00:00, 105.73it/s]


경로 분해 완료! 파일 저장됨: 경로분해_4.csv
